# Data preparation

#### Current format: 
Tokens each have an xml entry in their phonwords file. 
Accents each have an xml entry in their accents file.
Turns have are in their own files, and are recorded as time spans. The time span of a phonword is somewhere inside the time span of a turn.

Note: Each file is per-speaker, not per-turn or per-conversation.

#### Desired format:
One turn per line, whitespace-separated tokens, tab, binary string with 1 for nuclear pitch accent.

##### Find files:

In [57]:
import os

data_dir = '/afs/inf.ed.ac.uk/group/corpora/large/switchboard/nxt/xml'


dialog_num = 'sw2018'
users = ('A','B')
turn_files = [os.path.join(data_dir,'turns','.'.join([dialog_num,user,'turns','xml'])) for user in users]
acc_files = [os.path.join(data_dir,'accent','.'.join([dialog_num,user,'accents','xml'])) for user in users]
wd_files = [os.path.join(data_dir,'phonwords','.'.join([dialog_num,user,'phonwords','xml'])) for user in users]

In [149]:
import numpy as np
import xml.etree.ElementTree as ET
nite = '{http://nite.sourceforge.net/}'

accent_dict = {'nuclear':1,
               'plain':0,
               'pre-nuclear':0}

words = []
ids = []
times = []
wd_to_i = {}
i_to_wd = {}
id_to_acc = {}
counter = 0
for i,wd_file in enumerate(wd_files):
    tmp_wds = []
    tmp_ids = []
    tmp_times = []
    if os.path.exists(acc_files[i]): # only pay attention to ones that have accent files
        wd_tree = ET.parse(wd_file)
        wd_root = wd_tree.getroot()
        for phonword in wd_root.findall('phonword'):
            orth = phonword.attrib['orth']
            id_num = phonword.attrib[nite+'id']
            start_time = phonword.attrib[nite+'start']
            if not orth in wd_to_i:
                wd_to_i[orth] = counter
                i_to_wd[counter] = orth
                counter += 1
            tmp_wds.append(wd_to_i[orth])
            tmp_ids.append(id_num) # TODO since these ids are unique, I can make a lookup table for them too for speed
            tmp_times.append(start_time)
        words.append(tmp_wds)
        ids.append(tmp_ids)
        times.append(tmp_times)
                
        acc_tree = ET.parse(acc_files[i])
        acc_root = acc_tree.getroot()
        for acc in acc_root.findall('accent'):
            for chld in acc:
                acc_id = chld.attrib['href'].split('(')[-1][:-1]
                id_to_acc[acc_id]=accent_dict[acc.attrib['type']]
    else:
        print('no accent file found')
        
            
#print([i_to_wd[i] for i in words[:5]])
len(ids[0])

[['ms1A_pw1',
  'ms1A_pw2',
  'ms1A_pw3',
  'ms1A_pw4',
  'ms3A_pw5',
  'ms3A_pw6',
  'ms3A_pw7',
  'ms3A_pw8',
  'ms3A_pw9',
  'ms3A_pw10',
  'ms3A_pw11',
  'ms3A_pw12',
  'ms3A_pw13',
  'ms3A_pw14',
  'ms3A_pw15',
  'ms3A_pw16',
  'ms3A_pw17',
  'ms3A_pw18',
  'ms5A_pw19',
  'ms5A_pw20',
  'ms5A_pw21',
  'ms5A_pw22',
  'ms5A_pw23',
  'ms5A_pw24',
  'ms5A_pw25',
  'ms5A_pw26',
  'ms5A_pw27',
  'ms6A_pw28',
  'ms6A_pw29',
  'ms6A_pw30',
  'ms6A_pw31',
  'ms6A_pw32',
  'ms6A_pw33',
  'ms6A_pw34',
  'ms6A_pw35',
  'ms6A_pw36',
  'ms6A_pw37',
  'ms6A_pw38',
  'ms6A_pw39',
  'ms8A_pw40',
  'ms8A_pw41',
  'ms8A_pw42',
  'ms8A_pw43',
  'ms8A_pw44',
  'ms8A_pw45',
  'ms8A_pw46',
  'ms8A_pw47',
  'ms8A_pw48',
  'ms8A_pw49',
  'ms8A_pw50',
  'ms8A_pw51',
  'ms8A_pw52',
  'ms8A_pw53',
  'ms10A_pw54',
  'ms12A_pw55',
  'ms12A_pw56',
  'ms12A_pw57',
  'ms12A_pw58',
  'ms12A_pw59',
  'ms12A_pw60',
  'ms12A_pw61',
  'ms12A_pw62',
  'ms12A_pw63',
  'ms12A_pw64',
  'ms12A_pw65',
  'ms12A_pw66',
  'ms1

### Make np array of accents

In [157]:
words = (np.array(words[0]), np.array(words[1]))
times = (np.array(times[0]), np.array(times[1]))
words[1].shape

(856,)

In [158]:
accents = (np.zeros(words[0].shape),np.zeros(words[1].shape))

for i in (0,1):
    for j in range(words[i].shape[0]):
        id_num = ids[i][j]
        if id_num in id_to_acc:
            accents[i][j] = id_to_acc[id_num]
        else:
            accents[i][j] = 0
    
accents

(array([1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
        1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
        0., 

### Iterate through turns, writing the final form out turn by turn

In [162]:
turns0 = [(int(child.attrib[nite+'id'][1:]), (float(child.attrib[nite+'start']), float(child.attrib[nite+'end']),0)) for child in ET.parse(turn_files[0]).getroot()]
turns1 = [(int(child.attrib[nite+'id'][1:]), (float(child.attrib[nite+'start']), float(child.attrib[nite+'end']),1)) for child in ET.parse(turn_files[1]).getroot()]
turns = turns0 + turns1
turns.sort()
turns = dict(turns)

In [168]:
for num in turns:
    speaker = turns[num][2]
    start = turns[num][0]
    end = turns[num][1]
    turn = words[speaker][times[speaker] > start]
    
    

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'float'